In [10]:
import matplotlib.pyplot as plt

from tensorflow.keras.applications import InceptionV3
from keras.applications.inception_v3 import preprocess_input
from keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.layers import Dense, Activation, Flatten, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import SGD, Adam


img_width, img_height = 299, 299


epochs = 5
batch_size = 10
num_train_img = 900
num_val_img = 100

base_model = InceptionV3(weights='imagenet', 
                      include_top=False, 
                      input_shape=(img_height, img_width, 3))

In [3]:
train_dir = 'furniture_data/train'


train_datagen =  ImageDataGenerator(
      preprocessing_function=preprocess_input,
      rotation_range=90,
      horizontal_flip=True,
      vertical_flip=True
    )

train_generator = train_datagen.flow_from_directory(train_dir, 
                                                    target_size=(img_height, img_width), 
                                                    batch_size=batch_size)

Found 4480 images belonging to 3 classes.


In [8]:
val_dir = 'furniture_data/test'


val_datagen =  ImageDataGenerator(
      preprocessing_function=preprocess_input,
      rotation_range=90,
      horizontal_flip=True,
      vertical_flip=True
    )

val_generator = val_datagen.flow_from_directory(val_dir, 
                                                    target_size=(img_height, img_width), 
                                                    batch_size=batch_size)

Found 1170 images belonging to 3 classes.


In [9]:
def build_final_model(base_model, dropout, fc_layers, num_classes):
    for layer in base_model.layers:
        layer.trainable = True
        
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Flatten()(x)
        
    for fc in fc_layers:
        # new fc layer, random init
        x = Dense(fc, activation='relu')(x)
        x = Dropout(dropout)(x)
        
    # new softmax layer
    predictions = Dense(num_classes, activation='softmax')(x)
    
    final_model = Model(inputs=base_model.input, outputs=predictions)
    
    return final_model

classes = ['bed', 'chair', 'sofa']
fc_layers = [1024, 1024]
dropout = 0.2

model = build_final_model(base_model,
                          dropout=dropout,
                          fc_layers=fc_layers,
                          num_classes=len(classes)
                          )

In [14]:
adam = Adam(lr=0.00001)
model.compile(adam, loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(train_generator, epochs=epochs, workers=0, 
                                       steps_per_epoch=num_train_img // batch_size, 
                                       shuffle=True, validation_data=val_generator,
        validation_steps=num_val_img // batch_size)

Epoch 1/5
90/90 [==============================] - 324s 4s/step - loss: 0.9578 - accuracy: 0.5589 - val_loss: 0.7185 - val_accuracy: 0.7500
Epoch 2/5
12/90 [===>..........................] - ETA: 3:59 - loss: 0.7633 - accuracy: 0.7167

KeyboardInterrupt: 